In [26]:
from opensearchpy import OpenSearch

from opensearchpy import OpenSearch

client = OpenSearch(
    hosts = [{"host": "localhost", "port": 9200}],
    http_auth = ("admin", "admin"),
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)



In [29]:
index_name = "turism_madrid"

mapping = {
    "mappings": {
        "properties": {
            "main_etapa_details": {
                "type": "object"
            },
            "main_etapa_name": {
                "type": "text"
            },
            "route_details": {
                "type": "nested",
                "properties": {
                    "itinerario_details": {
                        "type": "nested",
                        "properties": {
                            "itinerario_name": {"type": "text"},
                            "itinerario_details": {"type": "object"}
                        }
                    },
                    "route_details": {"type": "object"},
                    "route_name": {"type": "text"},
                }
            }
        }
    }
}

client.indices.create(index=index_name, body=mapping)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'turism_madrid'}

In [ ]:
# define function to add itinerario_details to the index
def add_itinerario_details(main_etapa_name, route_name, itinerario_details):
    # check if main_etapa_name already exists
    main_etapa_query = {"query": {"term": {"main_etapa_name": main_etapa_name}}}
    main_etapa_res = client.search(index="my_index", body=main_etapa_query)
    
    # if main_etapa_name exists, check if route_name already exists
    if main_etapa_res["hits"]["total"]["value"] > 0:
        route_query = {
            "query": {
                "bool": {
                    "must": [
                        {"term": {"main_etapa_name": main_etapa_name}},
                        {"nested": {"path": "route_details", "query": {"term": {"route_name": route_name}}}}
                    ]
                }
            }
        }
        route_res = client.search(index="my_index", body=route_query)
        
        # if route_name exists, append itinerario_details to the existing array
        if route_res["hits"]["total"]["value"] > 0:
            doc_id = route_res["hits"]["hits"][0]["_id"]
            doc = client.get(index="my_index", id=doc_id)
            for route_detail in doc["_source"]["route_details"]:
                if route_detail["route_name"] == route_name:
                    route_detail["itinerario_details"].append(itinerario_details)
            client.update(index="my_index", id=doc_id, body={"doc": {"route_details": doc["_source"]["route_details"]}})
        # if route_name does not exist, create a new route_details object with itinerario_details
        else:
            route_details = {"route_name": route_name, "itinerario_details": [itinerario_details]}
            client.update(index="my_index", body={"query": {"term": {"main_etapa_name": main_etapa_name}},"script": {"source": "ctx._source.route_details.add(params.route_details)","lang": "painless",


In [27]:
client.info()

{'name': '4a13cb0ffc22',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': '-NffdS9oQGOAdVGRAdVdVQ',
 'version': {'distribution': 'opensearch',
  'number': '2.6.0',
  'build_type': 'tar',
  'build_hash': '7203a5af21a8a009aece1474446b437a3c674db6',
  'build_date': '2023-02-24T18:57:04.388618985Z',
  'build_snapshot': False,
  'lucene_version': '9.5.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [3]:

# Create a new index
index_name = "my_index"
body = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "description": {"type": "text"},
            "timestamp": {"type": "date"}
        }
    }
}
client.indices.create(index=index_name, body=body)

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement opensearchpy (from versions: none)
ERROR: No matching distribution found for opensearchpy

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
# Define the updated data for the document
updated_data = {"key": "new_value"}

# Update the document in the OpenSearch or Elasticsearch database
client.update(index=index, id=doc_id, body={"doc": updated_data})

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f6169000ee0>: Failed to establish a new connection: [Errno -2] Name or service not known) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f6169000ee0>: Failed to establish a new connection: [Errno -2] Name or service not known)